# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [93]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [94]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [95]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [96]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [97]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [98]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION =
            {
                'class' : 'SimpleStrategy',
                'replication_factor' : 1
            }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [99]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [100]:
## CREATING THE TABLES
try:
    # Q1 - Modelling data after Query 1, using Session_id and Items_in_session (As asked by the where) and User_ID and Song to make the data unique
    query = "CREATE TABLE IF NOT EXISTS PLAY_DATA_Q1 "
    query += '''(
        -- SESSION PROPERTIES
        SESSION_ID INT,
        
        -- USER PROPERTIES
        USER_ID INT,
        FIRST_NAME TEXT,
        LAST_NAME TEXT,
        GENDER TEXT,
        LEVEL TEXT,
        LOCATION TEXT,
        
        -- USER INTERACTION PROPERTIES
        ITEM_IN_SESSION INT,
        
        -- SONG PROPERTIES
        SONG TEXT,
        ARTIST TEXT,
        LENGTH FLOAT,
        
        -- PRIMARY KEY
        PRIMARY KEY (
            -- PARTITION KEY
            SESSION_ID,
            
            -- CLUSTERING COLUMNS
            ITEM_IN_SESSION,
            USER_ID,
            SONG
        ));
    '''
    session.execute(query)

    # Q2 - Modelling the table after Query 2, using the same combinations as Q1 but primary key chain follows UserID -> SessionID (asked) and Items_in_session (sorting) and song (to make the primary key stronger)
    query = "CREATE TABLE IF NOT EXISTS PLAY_DATA_Q2 "
    query += '''(
        -- SESSION PROPERTIES
        SESSION_ID INT,
        
        -- USER PROPERTIES
        USER_ID INT,
        FIRST_NAME TEXT,
        LAST_NAME TEXT,
        GENDER TEXT,
        LEVEL TEXT,
        LOCATION TEXT,
        
        -- USER INTERACTION PROPERTIES
        ITEM_IN_SESSION INT,
        
        -- SONG PROPERTIES
        SONG TEXT,
        ARTIST TEXT,
        LENGTH FLOAT,
        
        -- PRIMARY KEY
        PRIMARY KEY (
            -- PARTITION KEY
            USER_ID,
            
            -- CLUSTERING COLUMNS
            SESSION_ID,
            ITEM_IN_SESSION,
            SONG
        ));
    '''
    
    session.execute(query)
    
    # Q3 - Modelling after Query 3 that asks for SONG (partition key) and USER_ID, SESSION_ID AND ITEM_IN_SESSION Clustering columns to make primary key stronger.
    query = "CREATE TABLE IF NOT EXISTS PLAY_DATA_Q3 "
    query += '''(
        -- SESSION PROPERTIES
        SESSION_ID INT,
        
        -- USER PROPERTIES
        USER_ID INT,
        FIRST_NAME TEXT,
        LAST_NAME TEXT,
        GENDER TEXT,
        LEVEL TEXT,
        LOCATION TEXT,
        
        -- USER INTERACTION PROPERTIES
        ITEM_IN_SESSION INT,
        
        -- SONG PROPERTIES
        SONG TEXT,
        ARTIST TEXT,
        LENGTH FLOAT,
        
        -- PRIMARY KEY
        PRIMARY KEY (
            -- PARTITION KEY
            SONG,
            
            -- CLUSTERING COLUMNS
            USER_ID,
            SESSION_ID,
            ITEM_IN_SESSION
        ));
    '''
    
    session.execute(query)

except Exception as e:
    print(e)

In [101]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        try:
            # Q1
            query = 'INSERT INTO play_data_q1 (ARTIST, FIRST_NAME, GENDER, ITEM_IN_SESSION, LAST_NAME, LENGTH, LEVEL, LOCATION, SESSION_ID, SONG, USER_ID)'
            query += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

            # Q2
            query = 'INSERT INTO play_data_q2 (ARTIST, FIRST_NAME, GENDER, ITEM_IN_SESSION, LAST_NAME, LENGTH, LEVEL, LOCATION, SESSION_ID, SONG, USER_ID)'
            query += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

            # Q3
            query = 'INSERT INTO play_data_q3 (ARTIST, FIRST_NAME, GENDER, ITEM_IN_SESSION, LAST_NAME, LENGTH, LEVEL, LOCATION, SESSION_ID, SONG, USER_ID)'
            query += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
            
        except Exception as e:
            print(e)

#### SELECT STATEMENTS

In [102]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)
session.row_factory = pandas_factory

In [103]:
# Q1 Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
try:
    rows = session.execute("SELECT * FROM play_data_q1 WHERE SESSION_ID = 338 AND ITEM_IN_SESSION = 4")
except Exception as e:
    print(e)

rows._current_rows[['artist', 'song', 'length']]

artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312

In [104]:
# Q2 Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
try:
    rows = session.execute("SELECT * FROM play_data_q2 WHERE USER_ID = 10 AND SESSION_ID = 182")
except Exception as e:
    print(e)

rows._current_rows[['artist', 'song', 'item_in_session', 'first_name', 'last_name']]

artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

   item_in_session first_name last_name  
0                0     Sylvie      Cruz  
1                1     Sylvie      Cruz  
2                2     Sylvie      Cruz  
3                3     Sylvie      Cruz

In [105]:
# Q3 Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    rows = session.execute("SELECT * FROM play_data_q3 WHERE song = 'All Hands Against His Own'")
except Exception as e:
    print(e)

rows._current_rows[['first_name', 'last_name']]

first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson

### Drop the tables before closing out the sessions

In [106]:
# Dropping Table
try:
    query = "DROP TABLE play_data_q1;"
    rows = session.execute(query)
    query = "DROP TABLE play_data_q2;"
    rows = session.execute(query)
    query = "DROP TABLE play_data_q3;"
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [107]:
session.shutdown()
cluster.shutdown()